In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats

In [3]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation


In [4]:
LTS = pd.read_csv('listings.csv', delimiter = ',', header = 0)
NBH = pd.read_csv('neighbourhoods.csv', delimiter = ',', header = 0)
REW = pd.read_csv('reviews.csv', delimiter = ',', header = 0)
CAL = pd.read_csv('calendar.csv.gz', delimiter = ',', header = 0, low_memory=False)
LTS_GZ = pd.read_csv('listings.csv.gz', delimiter = ',', header = 0, low_memory=False)
REW_GZ = pd.read_csv('reviews.csv.gz', delimiter = ',', header = 0, low_memory=False)
NBH_J= pd.read_json('neighbourhoods.geojson', lines=True)

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [ ]:
####Begin Data Cleaning

In [6]:
# LTS.drop(columns=['neighbourhood_group','last_review'],inplace=True)
# NBH.drop(columns=['neighbourhood_group'],inplace=True)